#1. OBTENER LINK MASTER.JSON

Copiar archivo HTML en RAIZ y poner el nombre para obtener enlace master.json completo, si posee .mp4 no es necesario este proceso

In [27]:
#@title
#@markdown Archivo HTML
video_html =  "video.html" #@param {type: "string" }
# @ markdown ---

find1='{"url":"'
find2='","origin"'

#video_html = "video.html"
with open(video_html,"r") as file:
  contex = file.read()

print ('LINK A COPIAR\n')
master_json=contex.split(find1)[1].split(find2)[0]
master_json

LINK A COPIAR



'https://195vod-adaptive.akamaized.net/exp=1665045822~acl=%2Ff2165557-05f6-44f6-91eb-703d6fc1c1b2%2F%2A~hmac=a9806b7965745c75962c3730e05966b7932c88047d8d5158532f6760012d5074/f2165557-05f6-44f6-91eb-703d6fc1c1b2/sep/video/c5432c5b,2c312ef7,359f27f8,f303633c,fce14279/audio/5e95e76b,6b1cca76,ee476b48/master.json?query_string_ranges=1&base64_init=1'

#2. DESCARGA DE ARCHIVOS

Se ingresa el parametro
MASTER JSON, procede a descargar el audio y video en su mejor calidad (solo unos 2 minutos)

In [ ]:
#@title Enlace json de Vimeo a descargar
#!/bin/env python3

#import argparse
import subprocess
import requests

#    master_json_url = "https://91vod-adaptive.akamaized.net" \
#    "/exp=1668939686" \
#    "~acl=%2F842360cb-7711-4e48-9bd5-144e4686a2e2%2F%2A" \
#    "~hmac=a982f9ed2cc1e2fc7c61201f9997e0262917b13a89494c4169e4b3848d148f59/842360cb-7711-4e48-9bd5-144e4686a2e2" \
#    "/sep" \
#    "/video/e524c67d,336fea7d,5e87cb6d,306933cc,d42a911f" \
#    "/audio/ebb79ef9,7c03de6a,cad3266d" \
#    "/master.json?query_string_ranges=1&base64_init=1" \

#master_json_url = "https://91vod-adaptive.akamaized.net/exp=1668995241~acl=%2F842360cb-7711-4e48-9bd5-144e4686a2e2%2F%2A~hmac=9d08db3017c9c54bb1732988a82f58f114024eb14419590f5108bfe05e7d36a6/842360cb-7711-4e48-9bd5-144e4686a2e2/sep/video/e524c67d,336fea7d,5e87cb6d,306933cc,d42a911f/audio/ebb79ef9,7c03de6a,cad3266d/master.json?query_string_ranges=1&base64_init=1"
#output_file = "/content/drive/MyDrive/DINTA/Clase21/result.mp4"  # "./result.mp4"

#master_json_url = master_json
#@markdown Datos para procesar master.json
master_json_url =  PEGAR LINK AQUI #@param {type: "string" }
#sample "https://57vod-adaptive.akamaized.net/exp=1686297949~acl=%2F4bdd866d-6b55-4651-880a-91db624ac528%2F%2A~hmac=3e6cb7cb7f37fe6c056cb09b1a9e4e11c3ddf4a342b8fd8581cc4b269630e1b3/4bdd866d-6b55-4651-880a-91db624ac528/sep/video/2062e218,2b1f72ad,9a3d5e13,c3d65349,c4fcc39e/audio/84f68df9,ed4da030,faf555b2/master.json?base64_init=1\\u0026query_string_ranges=1" #@param {type: "string"}
#@markdown ---

#if len(sys.argv) > 1:
#     master_json_url = sys.argv[1]
#     output_file = sys.argv[2]
#else:
#    print('master.json must be passed as argument', file=sys.stderr)
#    exit(1)

# eliminanos de /SEP/ en adelante y agregamos /PARCEL/
base_url = master_json_url[:master_json_url.rfind('/sep/')]
#base_url = "https://117vod-adaptive.akamaized.net/exp=1668959455~acl=%2Fa78da094-44ed-4dc3-980a-19095f3b9acc%2F%2A~hmac=abf8ebfffc8c80493c37fd6cef039d13fd7371e53e72db5ba133abe20ba619f2/a78da094-44ed-4dc3-980a-19095f3b9acc/"
base_url = base_url + "/parcel/"

# obtenemos el MASTER.JSON
resp = requests.get(master_json_url)
content = resp.json()

# obtenemos la calidad mas alta de video
heights = [(i, d['height']) for (i, d) in enumerate(content['video'])]
idx, _ = max(heights, key=lambda h: h[1])
video = content['video'][idx]
video_base_url = base_url + video['base_url']
video_base_url = base_url + 'video/'
video_file =  video['id'] + '.mp4'
video_url = video_base_url + video_file
print('VIDEO URL:', video_url)

# Audio download here
bitrate = [(i, d['bitrate']) for (i, d) in enumerate(content['audio'])]

print('Bitrate', bitrate)

idx, _ = max(bitrate, key=lambda x: x[1])
audio = content['audio'][idx]
audio_base_url = base_url + audio['base_url']
audio_base_url = base_url + 'audio/'
audio_file = audio['id'] + '.mp4'
audio_url = video_base_url + audio_file
print('AUDIO URL:', audio_url)

print("Downloading Files ...")
with open("files.txt","w") as file:
    file.write(video_file+"\n")
    file.write(audio_file+"\n")

with open("urls.txt","w") as file:
    file.write(video_url+"\n")
    file.write(audio_url+"\n")

%time !wget -i urls.txt
print("Download Files!!!")

print( audio_file )
print( video_file )

print ("DOWNLOAD READY")

#3. MUX AUDIO/VIDEO EN MP4

Se ejecuta proceso para unir ambos archivos utilizando MKV Tool NIX, a diferencia del antiguo MIX realizado con FFMPEG, este darda solo segundos y no horas, ya que NO realiza CODE al obtener como resultado un archivo .MKV que solo une ambos archivos. Youtube acepta este formato, aunque le toma mas tiempo el proceso interno a youtube

In [ ]:
#@title Libreria MKV TOOL NIX
#@markdown
#@markdown Install MKVToolNix

import os
from IPython.display import clear_output

Version = "70.0.0" #@param {type:"string"}
downloadLink = f'https://mkvtoolnix.download/appimage/MKVToolNix_GUI-{Version}-x86_64.AppImage'

mkvpropedit = '/usr/local/bin/mkvpropedit'
mkvmerge = '/usr/local/bin/mkvmerge'
mkvextract = '/usr/local/bin/mkvextract'
mkvinfo = '/usr/local/bin/mkvinfo'

if os.path.exists(mkvpropedit) is True:
    !rm {mkvpropedit}
if os.path.exists(mkvmerge) is True:
    !rm {mkvmerge}
if os.path.exists(mkvextract) is True:
    !rm {mkvextract}
if os.path.exists(mkvinfo) is True:
    !rm {mkvinfo}

!sudo curl -L {downloadLink} -o /usr/local/bin/MKVToolNix_GUI.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI.AppImage /usr/local/bin/mkvpropedit
!sudo chmod a+rx /usr/local/bin/mkvpropedit
!sudo ln -s /usr/local/bin/MKVToolNix_GUI.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
!sudo ln -s /usr/local/bin/MKVToolNix_GUI.AppImage /usr/local/bin/mkvextract
!sudo chmod a+rx /usr/local/bin/mkvextract
!sudo ln -s /usr/local/bin/MKVToolNix_GUI.AppImage /usr/local/bin/mkvinfo
!sudo chmod a+rx /usr/local/bin/mkvinfo

clear_output()
print("Successfully Installed.")

Successfully Installed.


In [ ]:
#@title Combinar archivo audio con video
#@markdown Datos para Mux de Audio Video
#video_file =   #@param {type: "string"}
#audio_file = audio_file  #@param {type: "string"}
output_file = "output.mkv"  #@param {type: "string"}
#@markdown ---

#@title
#--track-name 1:video --language 1:spa --default-track-flag 1:yes
commandLine = f'--output {output_file} --no-audio --no-subtitles --no-attachments --no-track-tags --no-global-tags --no-chapters  --forced-display-flag 1:yes {video_file} --track-name 0:uno --language 0:spa --default-track-flag 0:yes --forced-display-flag 0:no {audio_file}  '
#!mkvmerge --output output.mkv --no-audio --no-subtitles --no-attachments --no-track-tags --no-global-tags --no-chapters  --track-name 1:video --language 1:spa --default-track-flag 1:yes --forced-display-flag 1:yes cfae1636.mp4 --track-name 0:uno --language 0:spa --default-track-flag 0:yes --forced-display-flag 0:no f5d223fd.mp4
print(commandLine)
!mkvmerge {commandLine}


#4. MONTAR GDRIVE

Se monta google Drive, para que el archivo resultante, quede disponible aun despues de la sesion Colab

In [ ]:
#@title Autorizar acceso a Google Drive
#@markdown Datos para Auth2
REQUERIR_DRIVE = False  #@param {type: "boolean"}
#@markdown ---

if (REQUERIR_DRIVE) :
   from google.colab import drive
   drive.mount('/content/drive')


#5. COPIAR VIDEO A DRIVE

Se realiza copia manualmente o arrastrando o descargando con click en los tres botones del archivo

In [ ]:
!cp output.mkv  /drive/..../archivo_video.mkv